In [1]:
#pip install xgboost

In [2]:
#pip install imblearn

In [3]:
import seaborn as sns
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler as RUS
from sklearn.datasets import make_classification as MC
from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import confusion_matrix as cm
from sklearn.decomposition import PCA
from sklearn.datasets import make_blobs as MB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)"]


### Import Data with Weighted Classes

In [ ]:
data = pd.read_csv('http://apmonitor.com/pds/uploads/Main/imbalanced_train.csv')
datat = pd.read_csv('http://apmonitor.com/pds/uploads/Main/imbalanced_test.csv')
data.columns = [0,1,2,3,4,'label']
datat.columns = [0,1,2,3,4,'label']
data['label'].hist(bins=5)
plt.show()
data.head()

### Visualize Data

In [ ]:
sns.pairplot(data,hue='label')
plt.show()

### Find and drop redundant columns

In [ ]:
sns.heatmap(data.corr())
plt.show()

In this case, none of the columns show great correlation so we will use all features

### Create Models
#### Make One Model Weighted, and One Unweighted

In order to "weight" a model, there are several approaches one can take. Oversampling the minority classes or undersampling the majority classes is one way to try to "fill" imbalanced data. When oversampling minority classes, typically methods such as SMOTE are used. SMOTE creates artificial samples of a class with features within the vector space of the other class samples. Another approach, which is not shown here, is to use the "weights" kwarg that many model classes have. These accept some coefficient array that tells the model to adjust itself to compensate for imbalances. Below, I reproportion the data to be the same quantity for each class by oversampling the minority and undersampling the majority.

In [ ]:
Xtrain = data[[0,1,2,3,4]]
ytrain = data['label']
Xtest = datat[[0,1,2,3,4]]
ytest = datat['label']
fa = np.count_nonzero(ytrain.values==2)*2

ov = {0:int(np.count_nonzero(ytrain.values==0)),\
      1:int(np.count_nonzero(ytrain.values==1)),\
      2:int(fa)} #oversample the smallest class
un = {0:fa,1:fa,2:fa} #sample the same number of samples for each class
                      # so the population is roughly equal
over = SMOTE(sampling_strategy=ov, random_state=4, k_neighbors = 4)
under = RUS(sampling_strategy=un, random_state=4)
model_weighted = xgb.XGBClassifier(use_label_encoder=False)
model_imb = xgb.XGBClassifier(use_label_encoder=False)
# model_sampled = 
model_imb.fit(Xtrain,ytrain)
steps = [ ('o',over),('u',under),('xgboost',model_weighted)] #
pipeline = Pipeline(steps)
pipeline.fit(Xtrain,ytrain)
# yp = pipline.predict(Xtest)
pass

### Visualize Results

In [ ]:
# print(f1(ytest, model_imb.predict(Xtest),average=None))
plot_confusion_matrix(model_imb,Xtest,ytest)
plt.title(f'No resampling, F1={f1_score(ytest,model_imb.predict(Xtest),average="weighted"):.3g}')
plt.show()
print(f'True quantities: 0:{np.count_nonzero(ytest.values==0)} ' + \
      '1:{np.count_nonzero(ytest.values==1)} ' + \
      '2:{np.count_nonzero(ytest.values==2)}')
# print(f1(ytest, pipeline.predict(Xtest),average=None))
plot_confusion_matrix(pipeline,Xtest,ytest)
plt.title(f'Resampled, F1={f1_score(ytest,pipeline.predict(Xtest),average="weighted"):.3g}')
plt.show()


### Conclusion

We can see the effects of resampling. As one would expect, the  model trained on the resampled data predicted the smaller classes more often than the model trained on the skewed data. Unfortunately, this led to more misclassification of the majority class. So, the implementation of oversampling must be used with prudence guided by field knowledge. 